<a href="https://colab.research.google.com/github/Krish34853/D1_OST_0729/blob/main/NEWCalorimeter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
from pymongo import MongoClient
import bcrypt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# -------------------------------------------------------
# MongoDB Connection (from Streamlit Secrets)
# -------------------------------------------------------
mongo_uri = st.secrets["mongo"]["uri"]
db_name = st.secrets["mongo"]["db_name"]
client = MongoClient(mongo_uri)
db = client[db_name]
users_collection = db["users"]

# -------------------------------------------------------
# Helper Functions
# -------------------------------------------------------
def create_user(user_id, name, username, password):
    """Admin: Create new user"""
    if users_collection.find_one({"username": username}):
        st.warning("⚠️ Username already exists!")
        return
    hashed_pw = bcrypt.hashpw(password.encode(), bcrypt.gensalt())
    users_collection.insert_one({
        "user_id": user_id,
        "name": name,
        "username": username,
        "password": hashed_pw,
        "role": "user"
    })
    st.success("✅ User created successfully!")

def validate_user(username, password):
    """Validate login credentials"""
    user = users_collection.find_one({"username": username})
    if user and bcrypt.checkpw(password.encode(), user["password"]):
        return user
    return None

# -------------------------------------------------------
# Streamlit App Config
# -------------------------------------------------------
st.set_page_config(page_title="🥗 Calorimeter", page_icon="🍱", layout="wide")

st.title("🍱 Calorimeter — Calories & Macronutrients Tracker")

# -------------------------------------------------------
# Login Section
# -------------------------------------------------------
if "role" not in st.session_state:
    st.subheader("🔐 Login Portal")
    role = st.selectbox("Login as:", ["User", "Admin"])
    username = st.text_input("Username")
    password = st.text_input("Password", type="password")

    if st.button("Login"):
        user = validate_user(username, password)
        if user:
            if role == "Admin" and username == "admin":
                st.session_state["role"] = "admin"
                st.success("✅ Logged in as Admin!")
            elif role == "User" and user["role"] == "user":
                st.session_state["role"] = "user"
                st.session_state["username"] = username
                st.session_state["name"] = user["name"]
                st.success(f"✅ Welcome, {user['name']}!")
            else:
                st.error("❌ Role mismatch!")
        else:
            st.error("❌ Invalid credentials!")

# -------------------------------------------------------
# Admin Page
# -------------------------------------------------------
if "role" in st.session_state and st.session_state["role"] == "admin":
    st.header("👨‍💼 Admin Dashboard — Create Users")

    user_id = st.text_input("User ID")
    name = st.text_input("Full Name")
    new_username = st.text_input("New Username")
    new_password = st.text_input("New Password", type="password")

    if st.button("Create User"):
        if all([user_id, name, new_username, new_password]):
            create_user(user_id, name, new_username, new_password)
        else:
            st.warning("⚠️ Please fill all fields!")

    st.markdown("---")
    st.subheader("📋 All Registered Users")
    all_users = list(users_collection.find({}, {"_id": 0, "password": 0}))
    st.dataframe(all_users)

# -------------------------------------------------------
# User Page — Calorimeter
# -------------------------------------------------------
if "role" in st.session_state and st.session_state["role"] == "user":
    st.header(f"👋 Welcome, {st.session_state['name']}!")
    st.subheader("🥗 Calories & Macronutrients Calculator")

    # Food Database
    food_data = {
        'Food': ['Rice', 'Apple', 'Egg', 'Chicken Breast', 'Milk', 'Bread', 'Banana'],
        'Calories_per_100g': [130, 52, 155, 165, 42, 265, 89],
        'Protein_per_100g': [2.7, 0.3, 13, 31, 3.4, 9, 1.1],
        'Carbs_per_100g': [28, 14, 1.1, 0, 5, 49, 23],
        'Fat_per_100g': [0.3, 0.2, 11, 3.6, 1, 3.2, 0.3]
    }
    food_df = pd.DataFrame(food_data)

    # Sidebar Inputs
    st.sidebar.header("🍽️ Enter Your Food Intake (grams)")
    st.sidebar.write("Use sliders below to record how much you ate:")
    quantities = {}
    for food in food_df['Food']:
        quantities[food] = st.sidebar.slider(f"{food} (g)", 0, 1000, 0, 10)

    # Merge with Quantities
    quantities_df = pd.DataFrame(list(quantities.items()), columns=['Food', 'Quantity_g'])
    merged_df = pd.merge(food_df, quantities_df, on='Food')

    # Calculate Nutrients
    for col in ['Calories_per_100g', 'Protein_per_100g', 'Carbs_per_100g', 'Fat_per_100g']:
        nutrient = col.replace('_per_100g', '')
        merged_df[nutrient] = (merged_df[col] * merged_df['Quantity_g']) / 100

    # Totals
    total_calories = merged_df['Calories'].sum()
    total_protein = merged_df['Protein'].sum()
    total_carbs = merged_df['Carbs'].sum()
    total_fat = merged_df['Fat'].sum()

    if merged_df['Quantity_g'].sum() > 0:
        st.subheader("🔥 Daily Summary")
        col1, col2, col3, col4 = st.columns(4)
        col1.metric("Calories (kcal)", f"{total_calories:.1f}")
        col2.metric("Protein (g)", f"{total_protein:.1f}")
        col3.metric("Carbs (g)", f"{total_carbs:.1f}")
        col4.metric("Fat (g)", f"{total_fat:.1f}")

        st.markdown("---")

        st.subheader("📊 Nutrient Breakdown per Food")
        st.dataframe(
            merged_df[['Food', 'Quantity_g', 'Calories', 'Protein', 'Carbs', 'Fat']]
            .round(2)
            .style.highlight_max(color="lightgreen", axis=0)
        )

        st.subheader("📈 Macronutrient Distribution")
        labels = ['Protein', 'Carbs', 'Fat']
        values = [total_protein, total_carbs, total_fat]
        fig, ax = plt.subplots(figsize=(6, 4))
        ax.plot(labels, values, marker='o', linewidth=3, color='mediumseagreen')
        ax.set_title("Macronutrient Breakdown (g)")
        ax.grid(True, linestyle='--', alpha=0.6)
        st.pyplot(fig)

        st.subheader("🍛 Calories per Food Item")
        fig2, ax2 = plt.subplots(figsize=(8, 4))
        ax2.bar(merged_df['Food'], merged_df['Calories'], color='coral', edgecolor='black')
        ax2.set_ylabel("Calories (kcal)")
        ax2.set_title("Calorie Contribution by Food")
        ax2.grid(axis='y', linestyle='--', alpha=0.6)
        st.pyplot(fig2)
    else:
        st.info("👈 Use the sliders in the sidebar to add your food intake!")

